In [18]:
import duckdb
import pandas as pd

con = duckdb.connect(':memory:')

In [17]:
# Raw data
query_raw_data = """
    SELECT *
    FROM 'data/orders_merchant.csv'
"""

result_raw = con.sql(query_raw_data).df()
print(result_raw. tail(15))

       order_id     shopper_id  merchant_id order_date
18           19           1234         9875 2022-12-17
19           20           1239         9885 2022-09-17
20           21           1239         9885 2022-12-07
21           22           1239         9886 2022-10-16
22           23           1240         9886 2022-12-01
23           24           1241         9887 2022-11-05
24           25           1242         9887 2022-12-05
25         None           None         <NA>        NaT
26  merchant_id  merchant_name         <NA>        NaT
27         9876     Merchant A         <NA>        NaT
28         9875     Merchant B         <NA>        NaT
29         9877     Merchant C         <NA>        NaT
30         9885     Merchant D         <NA>        NaT
31         9886     Merchant E         <NA>        NaT
32         9887     Merchant F         <NA>        NaT


As can be seen from above, the data source, orders_merchant.csv, contains a mix of both order records and merchant mapping information within a single file. Therefore, I used the following queries to separate these two types of data:

In [21]:
query_orders_data = """

    -- Merchant data and empty rows don't have 'order_date' so this query separates us the orders data

    SELECT
        order_id,
        shopper_id,
        CAST(merchant_id AS INT) AS merchant_id,
        order_date
    FROM 'data/orders_merchant.csv' 
    WHERE order_date IS NOT NULL 
"""

result_raw = con.sql(query_orders_data).df()
print(result_raw)

   order_id shopper_id  merchant_id order_date
0         1       1234         9876 2022-09-09
1         2       1235         9875 2022-09-10
2         3       1236         9877 2022-09-11
3         4       1238         9876 2022-10-13
4         5       1237         9876 2022-10-12
5         6       1239         9876 2022-10-14
6         7       1239         9876 2022-10-15
7         8       1240         9875 2022-11-16
8         9       1234         9875 2022-11-17
9        10       1235         9875 2022-11-18
10       11       1236         9875 2022-12-19
11       12       1236         9875 2022-12-20
12       13       1238         9876 2022-10-03
13       14       1238         9877 2022-12-03
14       15       1238         9876 2022-10-01
15       16       1236         9877 2022-09-11
16       17       1240         9875 2022-09-16
17       18       1238         9876 2022-10-01
18       19       1234         9875 2022-12-17
19       20       1239         9885 2022-09-17
20       21  

In [20]:
query_merchant_data = """

-- The merchant data (inside orders) contains NULL values in order_date and merchant_id (foreign key from orders table),
-- so we filter those out.  
-- One row also has a NULL order_id, which is excluded by applying TRY_CAST

  SELECT
        CAST(order_id AS INT) AS merchant_id,
        shopper_id AS merchant_name
    FROM 'data/orders_merchant.csv'
    WHERE order_date IS NULL
      AND merchant_id IS NULL
      AND TRY_CAST(order_id AS INT) IS NOT NULL
"""

result_raw = con.sql(query_merchant_data).df()
print(result_raw)

   merchant_id merchant_name
0         9876    Merchant A
1         9875    Merchant B
2         9877    Merchant C
3         9885    Merchant D
4         9886    Merchant E
5         9887    Merchant F
